In [1]:
import os
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from datetime import datetime, timedelta
import datajoint as dj

c:\Users\Organoid PC\anaconda3\envs\utah_organoids\lib\site-packages\datajoint\plugin.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
dj.config.load("dj_local_conf_worker.json")
dj.conn()

[2025-12-19 11:28:08,450][INFO]: DataJoint is configured from c:\Users\Organoid PC\Documents\GitHub\utah_organoids\dj_local_conf_worker.json
[2025-12-19 11:28:08,836][INFO]: DataJoint 0.14.4 connected to utah-worker@db.datajoint.com:3306


DataJoint connection (connected) utah-worker@db.datajoint.com:3306

In [4]:
from workflow.pipeline import frame

Module stfio is not installed. Make sure .abf files are converted to .pkl in Python 2.


In [5]:
from workflow.pipeline import culture, ephys

## Test frame.py

In [6]:
frame.TimeFrameParamset()

frame_param_idx Unique identifier for the frame parameter set,num_frames Number of frames to extract,min_per_frame Length of each frame in minutes
1,12,5
2,4,15
3,8,15


In [7]:
organoid_id = 'O09'
experiment_start_time = (culture.Experiment & f"organoid_id = '{organoid_id}'" & "drug_name = 'Control'").fetch1('experiment_start_time')
insert_key = {
    "organoid_id": organoid_id,
    "experiment_start_time": experiment_start_time,
    "start_boundary": experiment_start_time + timedelta(minutes=10),
    "end_boundary": experiment_start_time + timedelta(hours=6, minutes=10),
    "frame_param_idx": 1
}

In [8]:
frame.FrameSession.insert1(insert_key, skip_duplicates=True)
frame.FrameSession()

organoid_id e.g. O17,experiment_start_time,start_boundary Start datetime for analysis,end_boundary End datetime for analysis,frame_param_idx Reference to TimeFrameParamset
O09,2023-05-03 17:33:00,2023-05-03 17:43:00,2023-05-03 23:43:00,1


In [9]:
frame.FrameAnalysis.populate()

{'success_count': 0, 'error_list': []}

In [10]:
frame.FrameAnalysis.ActiveTimeFrames()

organoid_id e.g. O17,experiment_start_time,start_boundary Start datetime for analysis,end_boundary End datetime for analysis,frame_param_idx Reference to TimeFrameParamset,frame_start Start of active time frame,frame_end End of active time frame,frame_firing_rate Firing rates for each frame
O09,2023-05-03 17:33:00,2023-05-03 17:43:00,2023-05-03 23:43:00,1,2023-05-03 17:49:03,2023-05-03 17:54:03,3.71133
O09,2023-05-03 17:33:00,2023-05-03 17:43:00,2023-05-03 23:43:00,1,2023-05-03 17:57:03,2023-05-03 18:02:03,0.951333
O09,2023-05-03 17:33:00,2023-05-03 17:43:00,2023-05-03 23:43:00,1,2023-05-03 18:06:03,2023-05-03 18:11:03,2.874
O09,2023-05-03 17:33:00,2023-05-03 17:43:00,2023-05-03 23:43:00,1,2023-05-03 18:18:49,2023-05-03 18:23:49,3.212
O09,2023-05-03 17:33:00,2023-05-03 17:43:00,2023-05-03 23:43:00,1,2023-05-03 18:29:49,2023-05-03 18:34:49,1.83267
O09,2023-05-03 17:33:00,2023-05-03 17:43:00,2023-05-03 23:43:00,1,2023-05-03 19:14:49,2023-05-03 19:19:49,0.488
O09,2023-05-03 17:33:00,2023-05-03 17:43:00,2023-05-03 23:43:00,1,2023-05-03 19:39:49,2023-05-03 19:44:49,1.00067
O09,2023-05-03 17:33:00,2023-05-03 17:43:00,2023-05-03 23:43:00,1,2023-05-03 19:56:49,2023-05-03 20:01:49,0.98
O09,2023-05-03 17:33:00,2023-05-03 17:43:00,2023-05-03 23:43:00,1,2023-05-03 20:11:49,2023-05-03 20:16:49,0.778
O09,2023-05-03 17:33:00,2023-05-03 17:43:00,2023-05-03 23:43:00,1,2023-05-03 20:22:49,2023-05-03 20:27:49,0.784666


## Test ephys.Impedance

In [11]:
organoid_id = 'O09'
rawfile_key = (ephys.EphysRawFile & "parent_folder = 'O09-12_raw'").fetch("KEY")[0]
key = {
    **rawfile_key,
    "organoid_id": organoid_id
}

In [12]:
frame.ImpedanceFile.insert1(key, skip_duplicates=True)
frame.ImpedanceFile()

file_path path to the file relative to the root directory,organoid_id e.g. O17
O09-12_raw/processed_O9-12_230503_173303.rhd,O09


In [13]:
frame.ImpedanceMeasurements.populate()
frame.ImpedanceMeasurements()

file_path path to the file relative to the root directory,organoid_id e.g. O17,port_id Port ID of the Intan acquisition system
O09-12_raw/processed_O9-12_230503_173303.rhd,O09,A


In [14]:
frame.ImpedanceMeasurements.Channel()

file_path path to the file relative to the root directory,organoid_id e.g. O17,channel_idx channel index,channel_id channel id,impedance_magnitude in Ohms,impedance_phase in Degrees
O09-12_raw/processed_O9-12_230503_173303.rhd,O09,0,A-000,1722650.0,-70.0959
O09-12_raw/processed_O9-12_230503_173303.rhd,O09,1,A-001,2203330.0,-65.7797
O09-12_raw/processed_O9-12_230503_173303.rhd,O09,2,A-002,1527100.0,-68.827
O09-12_raw/processed_O9-12_230503_173303.rhd,O09,3,A-003,3357900.0,-61.5856
O09-12_raw/processed_O9-12_230503_173303.rhd,O09,4,A-004,1532740.0,-68.7674
O09-12_raw/processed_O9-12_230503_173303.rhd,O09,5,A-005,1606910.0,-61.865
O09-12_raw/processed_O9-12_230503_173303.rhd,O09,6,A-006,1660960.0,-59.6131
O09-12_raw/processed_O9-12_230503_173303.rhd,O09,7,A-007,1772860.0,-59.5467
O09-12_raw/processed_O9-12_230503_173303.rhd,O09,8,A-008,2059110.0,-67.4862
O09-12_raw/processed_O9-12_230503_173303.rhd,O09,9,A-009,1715980.0,-54.9938


## analysis.Coherence

In [15]:
key = (ephys.EphysSessionProbe & "organoid_id = 'O09'").fetch("KEY")[6]
ephys.EphysSessionProbe & key


organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,probe unique identifier for this model of probe (e.g. serial number),port_id,"used_electrodes list of electrode IDs used in this session (if null, all electrodes are used)"
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,Q983,A,=BLOB=


In [16]:
# frame.Coherence.populate(key)

In [17]:
frame.Coherence()

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,execution_duration Time taken to compute coherence (in minutes)
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,5.13655


In [18]:
frame.Coherence.Connectivity()

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,electrode_a Electrode in coherence calculation,electrode_b Electrode in coherence calculation,f Frequency values,coherence Coherence values between electrode A and B
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,0,1,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,0,2,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,0,3,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,0,4,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,0,5,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,0,6,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,0,7,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,0,8,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,0,9,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,0,10,=BLOB=,=BLOB=


In [19]:
frame.Coherence.Synchrony() & "electrode = '0'"

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,band_name,electrode Electrode index,f Frequency values,synchrony Coherence between electrode LFP and frequency band signal
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,alpha,0,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,beta,0,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,delta,0,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,gamma,0,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,highgamma1,0,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,highgamma2,0,=BLOB=,=BLOB=
O09,2023-05-03 17:33:00,0,2023-05-03 18:18:00,2023-05-03 18:33:00,theta,0,=BLOB=,=BLOB=


## analysis.FOOOF

In [20]:
organoid_id = 'O10'
probe_key = (ephys.EphysSession & f"organoid_id = '{organoid_id}'" & "session_type = 'lfp'").fetch("KEY")[11]
probe_key

{'organoid_id': 'O10',
 'experiment_start_time': datetime.datetime(2023, 5, 3, 17, 33),
 'insertion_number': 0,
 'start_time': datetime.datetime(2023, 5, 3, 20, 3),
 'end_time': datetime.datetime(2023, 5, 3, 20, 18)}

In [21]:
num_elec = (frame.NumElectrodesInside & f"organoid_id = '{organoid_id}'").fetch1('num_electrodes')
key = {
    **probe_key,
    "fooof_param_idx": 1,
    "start_freq": 2.0,
    "end_freq": 200.0,
    "analysis_electrodes": np.arange(num_elec),
}
key

{'organoid_id': 'O10',
 'experiment_start_time': datetime.datetime(2023, 5, 3, 17, 33),
 'insertion_number': 0,
 'start_time': datetime.datetime(2023, 5, 3, 20, 3),
 'end_time': datetime.datetime(2023, 5, 3, 20, 18),
 'fooof_param_idx': 1,
 'start_freq': 2.0,
 'end_freq': 200.0,
 'analysis_electrodes': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       dtype=int64)}

In [30]:
frame.FOOOFParamset()

fooof_param_idx Unique identifier for the FOOOF parameter set,"peak_width_limits Lower and upper bounds on peak widths in Hz. e.g. [1, 12]",max_n_peaks Maximum number of peaks the model can fit. e.g. 6,min_peak_height Minimum absolute height of a peak above the aperiodic component. e.g. 0.1,peak_threshold Relative threshold that candidate peaks must exceed to be included. e.g. 2.0,"aperiodic_mode Form of the aperiodic fit. e.g. 'fixed', 'knee'"
0,=BLOB=,6,0.1,2.0,fixed
1,=BLOB=,3,0.05,3.5,fixed


In [22]:
frame.FOOOFSession.insert1(key, skip_duplicates=True)
frame.FOOOFSession()

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,fooof_param_idx Unique identifier for the FOOOF parameter set,start_freq Start frequency for FOOOF fitting,end_freq End frequency for FOOOF fitting,analysis_electrodes List of electrodes to perform FOOOF analysis on (will be averaged)
O10,2023-05-03 17:33:00,0,2023-05-03 20:03:00,2023-05-03 20:18:00,1,2.0,200.0,=BLOB=


In [23]:
frame.FOOOFAnalysis.populate()

{'success_count': 0, 'error_list': []}

In [24]:
frame.FOOOFAnalysis()

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,fooof_param_idx Unique identifier for the FOOOF parameter set,start_freq Start frequency for FOOOF fitting,end_freq End frequency for FOOOF fitting,spec_param_idx Reference to SpectrogramParamset,plot Plot of FOOOF fit (as json),aperiodic_offset Aperiodic offset,aperiodic_knee Aperiodic knee,aperiodic_exponent Aperiodic exponent,peak_center_frequencies Center frequencies of detected peaks,peak_powers Power of detected peaks (above aperiodic fit),peak_bandwidths Bandwidths of detected peaks,error Error of FOOOF fit (RMSE),r_squared R^2 of FOOOF fit
O10,2023-05-03 17:33:00,0,2023-05-03 20:03:00,2023-05-03 20:18:00,1,2.0,200.0,2,=BLOB=,0.00693588,0.0,0.462432,=BLOB=,=BLOB=,=BLOB=,0.114963,0.704759


In [25]:
import plotly.io as pio
trace_plot = (frame.FOOOFAnalysis & key).fetch1("plot")

fig = pio.from_json(trace_plot)
fig.show()

## mua.PopulationBursts

In [26]:
organoid_id = 'O13'
ephys_key = (ephys.EphysSession & f"organoid_id = '{organoid_id}'").fetch("KEY")[12]
key = {
    **ephys_key,
    "burst_param_idx": 1,
}

key

{'organoid_id': 'O13',
 'experiment_start_time': datetime.datetime(2023, 5, 25, 19, 18),
 'insertion_number': 0,
 'start_time': datetime.datetime(2023, 5, 25, 21, 48),
 'end_time': datetime.datetime(2023, 5, 25, 22, 3),
 'burst_param_idx': 1}

In [27]:
frame.BurstSession.insert1(key, skip_duplicates=True)
frame.BurstSession()

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,burst_param_idx Unique identifier for the burst detection parameter set
O13,2023-05-25 19:18:00,0,2023-05-25 19:18:00,2023-05-25 19:33:00,1
O13,2023-05-25 19:18:00,0,2023-05-25 21:48:00,2023-05-25 22:03:00,1


In [28]:
frame.PopulationBursts.populate()

{'success_count': 0, 'error_list': []}

In [29]:
frame.PopulationBursts()

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,burst_param_idx Unique identifier for the burst detection parameter set,burst_indices Indices of detected bursts within the time frame,burst_peak_heights Peak heights of detected bursts,burst_bounds Start and end indices of detected bursts (firing rate >= 10% of peak height),burst_spike_array Single electrode spike array for each burst (num_bursts x num_electrodes x time_window)
O13,2023-05-25 19:18:00,0,2023-05-25 19:18:00,2023-05-25 19:33:00,1,=BLOB=,=BLOB=,=BLOB=,=BLOB=
O13,2023-05-25 19:18:00,0,2023-05-25 21:48:00,2023-05-25 22:03:00,1,=BLOB=,=BLOB=,=BLOB=,=BLOB=
